# 2x Single-Image Super-Resolution on Grayscale Images

**Assignment:** Individual Class Project<br>
**Author:** Richard Hemphill<br>
**ID:** 903877709<br>
**Class:** ECE5268 Theory of Neural Networks<br>
**Instructor:** Dr. Georgios C. Anagnostopoulos<br>
**Description:** Using small-sized grayscale images, construct a CNN-based architecture that will downscale (magnify) the images by a factor of 2.<br>
**Emphasis:** Describe the concept of single-image super-resolution, describe the architecture in sufficient detail and show indicative training and post-training results.<br>


**References:**
* https://www.kaggle.com/spaceengineer1/alexonly-greyscale
* https://www.kaggle.com/c/two-sigma-financial-news/discussion/83593
* https://scikit-image.org/docs/dev/auto_examples/transform/plot_rescale.html

In [120]:
# Imports
import os.path
from pathlib import Path
from zipfile import ZipFile
import matplotlib.image as mpimg
from skimage.transform import resize
from kaggle.api.kaggle_api_extended import KaggleApi

In [121]:
# Constants
IMAGE_SET_OWNER = 'spaceengineer1'
IMAGE_SET_FILE = 'alexonly-greyscale'
RAW_IMAGE_FOLDER = 'images'
PROCESSED_IMAGE_FOLDER ='dataSet'
IMAGE_EXTENSION = 'jpg'
ORIGINAL_IMAGE_SIZE = 256
ORIGINAL_IMAGE_SHAPE = (ORIGINAL_IMAGE_SIZE, ORIGINAL_IMAGE_SIZE)
UPSCALE_FACTOR = 2
LOW_RES_IMAGE_SHAPE = ORIGINAL_IMAGE_SHAPE/UPSCALE_FACTOR

In [122]:
# Common Clases
class Color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## Prepocessing

In [123]:
# Connect to the Kaggle Database
def KaggleLogin():
    try:
        api = KaggleApi()
        api.authenticate()
        return api
    except:
        print('{}{}Need Kaggle Account{}'.format(Color.BOLD,Color.RED,Color.END))
        print('Copy Kaggle API Token to ~/.kaggle'.format())

In [124]:
# Extract raw image set
def DownloadImageSet(api, imageSetOwner = IMAGE_SET_OWNER, imageSetFile = IMAGE_SET_FILE):
    if not os.path.isfile(imageSetFile):
        api.dataset_download_files('{}/{}'.format(imageSetOwner, imageSetFile))
        zf = ZipFile('{}.zip'.format(imageSetFile))
        zf.extractall() 
        zf.close()

In [125]:
# Convert images as needed for training
def PreProcessImages(rawFolder = RAW_IMAGE_FOLDER, processedFolder=PROCESSED_IMAGE_FOLDER, shape=ORIGINAL_IMAGE_SHAPE, extension = IMAGE_EXTENSION):
    pathlist = Path(rawFolder).glob('**/*.{}'.format(extension))
    for rawImg in pathlist:
        img = mpimg.imread(rawImg)
        resizedImg = resize(img, shape, anti_aliasing=True)
        newImg = str(rawImg).replace(rawFolder, processedFolder)
        os.makedirs(os.path.dirname(newImg), exist_ok=True)
        mpimg.imsave(newImg, resizedImg)

In [126]:
# Pre Process Images
kaggle = KaggleLogin()
if kaggle is not None:
    DownloadImageSet(kaggle)
PreProcessImages()

## Create Model